**Imports & Data Load**

In [36]:
import re
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack

df = pd.read_csv("simplyhired_final_cleaned.csv")

**Normalize Titles**

In [37]:
def normalize_title(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s).lower()
    s = re.sub(r"\(.*?\)", " ", s)          # remove (...) like (Toronto)
    s = re.sub(r"\[.*?\]", " ", s)          # remove [...] like [Due Dilligence]
    s = re.sub(r"\b[a-z]*\d{3,}\b", " ", s) # remove codes like B3617, 1049532
    s = re.sub(r"[/|,;:\-\u2013\u2014]+", " ", s)
    s = re.sub(r"\b(i|ii|iii|iv|v)\b", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["title_norm"] = df["title"].map(normalize_title)

**Create updated titles column**

In [38]:
CANONICAL_PATTERNS = [
    ("Data Scientist", r"\bdata scientist\b"),

    ("ML Scientist/Research",
     r"\bmachine learning scientist\b|\bml scientist\b|\bresearch scientist\b|\bmachine learning researcher\b|"
     r"\bresearch engineer\b|\bscientist\b|\bmachine learning specialist\b|\bresearcher\b"),

    ("Machine Learning Engineer",
     r"\bmachine learning engineer\b|\bml engineer\b|\bai engineer\b"),

    # SPECIAL OVERRIDE: treat "Data & Analytics ... Analytics Engineer" as Data Engineer
    ("Data Engineer",
    r"\bdata engineer\b|"
    r"\bdata engineering\b|"
    r"\bdata\s*(?:&|and)\s*analytics\b.*\banalytics\b.*\bengineer\b|"
    r"\bdata\s*(?:&|and)\s*model\s*ops\b.*\bengineer\b"),

    # Analytics Engineer (general)
    ("Analytics Engineer",
     r"\banalytics(?:\s+\w+){0,3}\s+engineer\b|\bsenior analytics engineer\b|\bintermediate analytics engineer\b|\bai analytics\b|"
     r"\banalytics engineering\b"),

    ("Data Engineer", r"\bdata engineer\b"),

    ("Business Intelligence", r"\bbusiness intelligence\b|\bpower bi\b|\binsights\b"),
    ("Data Analyst", r"\bdata analyst\b|\bdata analytics\b"),
    ("Analyst", r"\banalyst\b|\brisk analytics\b|\brisk modelling\b"),
]

def updated_title(title_norm: str) -> str:
    t = title_norm
    for canon, pat in CANONICAL_PATTERNS:
        if re.search(pat, t):
            return canon
    # If not categorized, keep original normalized title
    return t


df["title_updated"] = df["title_norm"].map(updated_title)

df[["title", "title_updated"]].head(20)

,title,title_updated
0,Data Analyst,Data Analyst
1,YouTube Data Analyst,Data Analyst
2,Data Scientist,Data Scientist
3,AI/Machine Learning Data Engineer,Data Engineer
4,Data Product Analyst,Analyst
5,Data Product Analyst,Analyst
6,Data Science Engineer,data science engineer
7,Analytics Consultant,analytics consultant
8,Data Engineering Specialist,Data Engineer
9,"Senior Machine Learning Engineer, AI Platform",Machine Learning Engineer


**Vectorize title_updated (word + char TF-IDF)**

In [39]:
MIN_DF = 2

vec_word = TfidfVectorizer(analyzer="word", ngram_range=(1, 3), min_df=MIN_DF)
Xw = vec_word.fit_transform(df["title_updated"])

vec_char = TfidfVectorizer(analyzer="char_wb", ngram_range=(3, 5), min_df=MIN_DF)
Xc = vec_char.fit_transform(df["title_updated"])

X = hstack([Xw, Xc])
X.shape

(617, 1088)

**Cluster title_updated**

In [40]:
DISTANCE_THRESHOLD = 0.45  # adjust 0.35..0.55

S = cosine_similarity(X)
D = 1.0 - S

clusterer = AgglomerativeClustering(
    metric="precomputed",
    linkage="average",
    distance_threshold=DISTANCE_THRESHOLD,
    n_clusters=None
)

df["cluster_id"] = clusterer.fit_predict(D)

df["cluster_id"].value_counts().head(25)

cluster_id
4     233
2     124
0      58
38     41
16     37
19     33
11     11
1       9
6       8
8       4
10      4
9       3
21      3
14      2
22      2
26      2
25      2
12      2
29      2
13      2
3       2
28      2
5       2
30      2
18      2
Name: count, dtype: int64

**Inspect biggest clusters**

In [41]:
TOP_TITLES_PER_CLUSTER = 15
cluster_sizes = df["cluster_id"].value_counts()

for cid in cluster_sizes.head(15).index:
    print("\n" + "=" * 40)
    print(f"Cluster {cid} | size={cluster_sizes[cid]}")
    # show original titles inside the cluster
    print(df.loc[df["cluster_id"] == cid, "title"].value_counts().head(TOP_TITLES_PER_CLUSTER).to_string())


Cluster 4 | size=233
title
Data Analyst                                             15
Senior Data Analyst                                       6
Business Data Analyst                                     5
database analyst                                          4
Senior Financial Analyst                                  3
Data Product Analyst                                      2
Senior Analyst, Data Sciences                             2
HR Data & Analytics Analyst                               2
Senior Associate, Data Analyst                            2
Senior Analyst, Credit, ERM                               2
Pricing Analyst                                           2
Fraud Analyst (12-month contract)                         2
Sr Analyst, Measurement & Reporting                       2
Analyst, Supply Chain Replenishment - 1 Year Contract     2
Data Analyst, Child Health Eval Sciences                  2

Cluster 2 | size=124
title
Data Scientist                              

In [42]:
# --------------------------------------------------
# Print titles for clusters NOT in the top 15 largest clusters
# --------------------------------------------------

TOP_TITLES_PER_CLUSTER = 15
TOP_K = 15  # top clusters that you already print elsewhere

# Cluster sizes
cluster_sizes = df["cluster_id"].value_counts()

# The top 15 cluster IDs (the ones you printed)
top15_clusters = cluster_sizes.head(TOP_K).index.tolist()

print("=== TOP 15 CLUSTERS (excluded) ===")
print(cluster_sizes.head(TOP_K).to_string())

# Everything NOT in those top 15 clusters
remaining = df[~df["cluster_id"].isin(top15_clusters)].copy()

remaining_cluster_sizes = remaining["cluster_id"].value_counts()

print("\n=== CLUSTERS OUTSIDE TOP 15 ===")
print("Remaining rows:", len(remaining))
print("Remaining unique clusters:", remaining["cluster_id"].nunique())

print("\n=== Remaining cluster sizes (largest first) ===")
print(remaining_cluster_sizes.head(30).to_string())

# Print each remaining cluster and what's inside
print("\n=== CONTENTS OF CLUSTERS OUTSIDE TOP 15 ===")
for cid in remaining_cluster_sizes.index:
    sub = remaining[remaining["cluster_id"] == cid]

    print("\n" + "=" * 40)
    print(f"Cluster {cid} | size={len(sub)}")

    # show original titles inside the cluster
    print(sub["title"].value_counts().head(TOP_TITLES_PER_CLUSTER).to_string())

=== TOP 15 CLUSTERS (excluded) ===
cluster_id
4     233
2     124
0      58
38     41
16     37
19     33
11     11
1       9
6       8
8       4
10      4
9       3
21      3
14      2
22      2

=== CLUSTERS OUTSIDE TOP 15 ===
Remaining rows: 45
Remaining unique clusters: 30

=== Remaining cluster sizes (largest first) ===
cluster_id
26    2
25    2
12    2
29    2
13    2
3     2
28    2
5     2
30    2
18    2
20    2
7     2
36    2
15    2
17    2
31    1
23    1
37    1
33    1
34    1
43    1
32    1
41    1
44    1
27    1
40    1
42    1
39    1
35    1
24    1

=== CONTENTS OF CLUSTERS OUTSIDE TOP 15 ===

Cluster 26 | size=2
title
GIS Data and Analytics Technologist                                             1
GIS Data and Analytics Technologist for Planning Job Details | Richmond Hill    1

Cluster 25 | size=2
title
eCommerce Specialist    1
Ecommerce Specialist    1

Cluster 12 | size=2
title
Associate, Equities (Software)                 1
Investment Associate, Equities 

Review specific clusters

In [55]:
select_cluster_id=30

df[df['cluster_id'] == select_cluster_id]['title'].unique()


<StringArray>
['Pricing Analytics Architect - Fintech foundation (100% Remote - Canada)', 'Data Architect']
Length: 2, dtype: str